## Test Case 2: Hadley-like Meridonal Circulation

### Import Libraries

In [1]:
import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d.axes3d import Axes3D

# Physical Constants

In [2]:
a = 6.37122*(10**6)  # radius of the Earth (m)
g = 9.80616          # gravitational constant (m s^(-2))
p0 = 100000.0        # reference surface pressure (Pa)
cp = 1004.5          # specific heat capacity of dry air (J kg^(-1) K^(-1))
Rd = 287.0           # gas constant for dry air (J kg^(-1) K^(-1))
kappa = Rd/cp        # ratio of Rd to cp
T0 = 300.0           # isothermal atmosphere temperature (K)
pi = np.pi           # pi
htop = 12000.0

### Simulation Parameterizations

### Test Case Parameterizations

In [3]:
tau = 86400      # test case duration (s)
K = 5            # Number of overturning cells
u0 = 40          # reference zonal velocity
w0 = .15         # reference horizontal velocity

z_1 = 2000       # Lower boundary of tracer layer
z_2 = 5000       # Upper boundary of tracer layer

### Coordinate Transformations

The wind fields look nasty in cartesian so I prefer to just calculate them in shperical then go back to cartesian

In [4]:
# convert cartesian to spherical
def c2s(x,y,z):

    ############ calculate r
    r = np.sqrt(x**2 + y**2 + z**2)

    ############ calculate lambda
    lmbda = np.arctan2(y,x)
    
    # map to lambda in [0,2pi)
    lmbda = np.where(y < 0, lmbda + 2*pi, lmbda)

    ############ calculate phi
    phi = np.arctan2(z,np.sqrt(x**2 + y**2))
    
    return r,lmbda,phi



# convert spherical to cartesian
def s2c(r,lmbda,phi):
    
    x = r*np.cos(phi)*np.cos(lmbda)
    y = r*np.cos(phi)*np.sin(lmbda)
    z = r*np.sin(phi)
    
    return [x,y,z]

### Density and Pressure Field

The pressure field for this test case is given by

$$p(r) = p_0 e^{-g\frac{r-a}{R_{d}T_{0}}}$$

which is the hydrostatic pressure of an isothermal atmosphere at $T=T_0$. The perscribed density field is given by

$$\rho(r) = \rho_{0}e^{-(r-a)/H},\quad \text{where } \rho_{0}=\frac{p_{0}}{R_{d}T_{0}} \text{ and } H = \frac{R_{d}T_{0}}{g}$$



In [5]:
# Pressure field
def p(r):
    return p0*np.exp(-g*((r-a)/(Rd*T0)))

# Perscribed density field
def rho(r):
    return (p0/(Rd*T0))*np.exp(-(r-a)*(g/(Rd*T0)))

### Wind Field

In [7]:
def u_r_t(r,lmbda,phi,t):
    
    return ((w0*rho(a))/(K*rho(r))) * ((-2)*np.sin(K*phi)*np.sin(phi) + K*np.cos(K*phi)*np.cos(phi)) * np.sin(pi*(r-a)/htop)*np.cos(pi*t/tau)


In [8]:
def u_lmbda_t(phi):
    
    return u0*np.cos(phi)

def u_phi_t(r,lmbda,phi,t):
    
    return -((a*w0*pi*rho(a))/(K*htop*rho(r)))*np.cos(phi)*np.sin(K*phi)*np.cos(pi*(r-a)/htop)*np.cos(pi*t/tau)


##### Converting to Cartesian Wind Components

We then convert to cartesian by

$$\bar{u} = u_r\hat{r} + u_\lambda\hat{\lambda} + u_\phi\hat{\phi}$$

$$\Rightarrow \bar{u} = u_r(\cos{\phi}\cos{\lambda}\hat{x}+\cos{\phi}\sin{\lambda}\hat{y}+\sin{\phi}\hat{z}) + u_\lambda(\sin{\lambda}\hat{x}+\cos{\lambda}\hat{y}) + u_\phi(-\sin{\phi}\cos{\lambda}\hat{x}-\sin{\phi}\sin{\lambda}\hat{y}+\sin{\phi}\hat{z})$$

$$\Rightarrow \bar{u} = (u_r\cos{\phi}\cos{\lambda}+u_\lambda\sin{\lambda}-u_\phi\sin{\phi}\cos{\lambda})\hat{x} + (u_r\cos{\phi}\sin{\lambda}+u_\lambda\cos{\lambda}-u_\phi\sin{\phi}\sin{\lambda})\hat{y} + (u_r+u_\phi)\sin{\phi}\hat{z}$$

In [ ]:
def u_t(r,lmbda,phi,t):
    
    u_r = u_r_t(r,lmbda,phi,t)
    u_lmbda = u_lmbda_t(phi)
    u_phi = u_phi_t(r,lmbda,phi,t)
    
    u_x = np.cos(phi)*np.cos(lmbda)*u_r + (-np.sin(lmbda)*u_lmbda) + (-np.sin(phi)*np.cos(lmbda)*u_phi)
    u_y = np.cos(phi)*np.sin(lmbda)*u_r + np.cos(lmbda)*u_lmbda + (-np.sin(phi)*np.sin(lmbda)*u_phi)
    u_z = np.sin(phi)*u_r + np.cos(phi)*u_phi
    
    return np.stack((u_x,u_y,u_z))
    

## Tracer Test Cases

In [10]:
def get_q_init(r,lmbda,phi):
    
    z_0 = (z_1 + z_2)/2
    
    q = (1 + np.cos((2*pi*((r-a)-z_0))/(z_2 - z_1)))/2
    
    i1 = int(np.ceil(z_1/h)+depth)
    i2 = int(np.floor(z_2/h)+depth+1)
    
    q[:,:i1] = 0.0
    q[:,i2:] = 0.0
    
    return q

In [11]:
# fig = plt.figure(figsize=(20,20))
# ax = fig.add_subplot(2,2,1)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_lmbda_t(rr[0],rr[1],rr[2],0,1,0)[:,6])
# ax = fig.add_subplot(2,2,2)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_lmbda_t(rr[0],rr[1],rr[2],0,0,1)[:,6])
# ax = fig.add_subplot(2,2,3)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_phi_t(rr[0],rr[1],rr[2],0)[:,6])
# ax = fig.add_subplot(2,2,4)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_r_t(rr[0],rr[1],rr[2],0)[:,6])

In [12]:
# print(np.max(u_lmbda_t(rr[0],rr[1],rr[2],0,1,0)[:,6]))
# print(np.max(u_lmbda_t(rr[0],rr[1],rr[2],0,0,1)[:,6]))
# print(np.max(u_phi_t(rr[0],rr[1],rr[2],0)[:,6]))
# print(np.max(u_r_t(rr[0],rr[1],rr[2],0)[:,6]))